In [1]:
import findspark
findspark.init()

In [2]:
# from pyspark import SparkContext, SQLContext

# sc = SparkContext()
# sqlContext = SQLContext(sc)
# df = sqlContext.read.format('csv').options(header='true', inferschema='true',sep=";").load('data/2023-12-16.csv')
# df.show()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, from_unixtime

In [4]:
def load_parquet_files(spark, input_path):
    # Wczytaj dane z plików Parquet
    df = spark.read.parquet(input_path)
    return df

In [5]:
# Ścieżki do danych w HDFS
top_tracks_base_path = "data/parquet/"
spotify_songs_path = "data/spotify_songs.parquet"

In [6]:
# Inicjalizacja sesji Spark
spark = SparkSession.builder.appName("SpotifyAnalysis").getOrCreate()

23/12/29 16:01:17 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
23/12/29 16:01:17 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/29 16:01:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# Wczytaj dane z plików Parquet
top_tracks_df = load_parquet_files(spark, top_tracks_base_path)

spotify_songs_df = spark.read.parquet(spotify_songs_path)

In [8]:
# Analiza top 50 utworów
top_tracks_df.show(truncate=False)
# Wybierz ostatnie 5 kolumny i wyświetl
spotify_songs_df.select(spotify_songs_df.columns[-5:]).show()

# # Analiza najpopularniejszych wykonawców
# top_artists_df = top_tracks_df.groupBy("artist").count().orderBy(col("count").desc())
# top_artists_df.show()

# # Analiza cech utworów
# features_df = spotify_songs_df.select(
#     "danceability", "energy", "loudness", "speechiness", "acousticness",
#     "instrumentalness", "liveness", "valence", "duration_ms"
# )
# features_summary = features_df.summary()
# features_summary.show()

# # Wizualizacja cech utworów (przykład dla danceability)
# import matplotlib.pyplot as plt
# import pandas as pd

# danceability_data = features_df.select("danceability").toPandas()
# danceability_data.plot(kind="hist", bins=20, title="Distribution of Danceability")
# plt.show()

# Zamknięcie sesji Spark

+----------------------+----+-------------------------------------------+---------------+-------------------+----------+
|track_id              |rank|track_name                                 |artist         |date_added         |popularity|
+----------------------+----+-------------------------------------------+---------------+-------------------+----------+
|06qMRF18gwbOYYbnP2du6i|1   |Last Christmas - Single Version            |Wham!          |2023-12-25 08:54:00|89        |
|0bYg9bo50gSsH3LtXe2SQn|2   |All I Want for Christmas Is You            |Mariah Carey   |2023-12-25 08:54:00|100       |
|2EjXfH91m7f8HiJN1yQg97|3   |Rockin' Around The Christmas Tree          |Brenda Lee     |2023-12-25 08:54:00|98        |
|2TE4xW3ImvpltVU0cPcKUn|4   |Merry Christmas Everyone                   |Shakin' Stevens|2023-12-25 08:54:00|91        |
|7vQbuQcyTflfCIOu3Uzzya|5   |Jingle Bell Rock                           |Bobby Helms    |2023-12-25 08:54:00|94        |
|7uoFMmxln0GPXQ0AcCBXRq|6   |Sno

In [9]:
# Wybierz tylko kolumnę 'track_id' z obu ramek danych
top_tracks_ids = top_tracks_df.select("track_id")
spotify_songs_ids = spotify_songs_df.select("track_id")

# Użyj metody subtract, aby znaleźć różnicę między dwoma ramkami
missing_tracks = top_tracks_ids.subtract(spotify_songs_ids)

# Oznacz w top_tracks rekordy, których brakuje w spotify_songs
missing_tracks_list = missing_tracks.rdd.map(lambda x: x[0]).collect()
top_tracks_df = top_tracks_df.withColumn("missing", col("track_id").isin(missing_tracks_list))

In [10]:
# Pokaż rekordy z top tracks, których brakuje
top_tracks_df[top_tracks_df['missing']==True].show(truncate=False)

+----------------------+----+----------------+--------------+-------------------+----------+-------+
|track_id              |rank|track_name      |artist        |date_added         |popularity|missing|
+----------------------+----+----------------+--------------+-------------------+----------+-------+
|4NGn7ZjI1Ex9ZjcbOa6KNt|25  |Rolly           |ReTo          |2023-12-29 09:09:03|28        |true   |
|61rJpXPZ4wkqmdsLYJu6LK|48  |Z Tobą na chacie|Daria Zawiałow|2023-12-29 09:09:03|63        |true   |
|4uayCD0blZhblTiQqKLYiC|49  |Skippers        |Young Leosia  |2023-12-29 09:09:03|55        |true   |
+----------------------+----+----------------+--------------+-------------------+----------+-------+



In [12]:
spark.stop()